# GMM

Note that here testing is done on .wav files. For testing on spectrogram , the readDir function can be changed accordingly. However here too, the MFCC features are calculated from decibel power spectrum only.

In [1]:
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from numpy import log, dot, e
import librosa
from numpy.random import rand
import tensorflow as tf
from tensorflow import keras

import tensorflow.keras as keras

In [2]:
def predict_framewise(model,x_test):
        '''
        Framewise classification (speech or music)
        
        '''
        
        
        lis=[]   

        
        for i in range(0,len(x_test)):
            k=0
            
            temp =[]
            #print(x_test[i].shape)

            clusters = model.predict(x_test[i])
             
            
            for i in range(len(clusters)):
                if(clusters[i]==1):
                    temp.append([0,1])
                else:
                    temp.append([1,0])
            pred=np.array(temp)

            
                
            
            lis.append(pred)

        ### WRITE YOUR CODE HERE - 5 MARKS

        
        
        return lis

In [3]:
def predict_aggregate(y_pred_framewise):
        '''
        Aggregate frames to give a single class label (music or speech) to the entire audio file
        
            
        '''
       

        
        y_hat= np.ones((1,2))
        for i in range (0,len(y_pred_framewise)):
            
            zero=0
            one=0
            t=y_pred_framewise[i]
            for j in range(0, len(t)):
                
                if(t[j][0] == 1):
                    one+=1
                else:
                    zero+=1
            if(one>zero):
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([1,0] , (-1,1) ) ) )) 
                
            else:
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([0,1] , (-1,1) ) ) ))  

        ### WRITE YOUR CODE HERE - 5 MARKS

        y_hat=y_hat[1:]
        return y_hat

In [4]:
def splitData(X, t, testFraction=0.2, randomize = False):
    """
    Split the data randomly into training and test sets
    Use numpy functions only
    Inputs:
        X: (np array of len Nclips) input feature vectors
        t: (np array of len Nclips) targets; one hot vectors
        testFraction: (float) Nclips_test = testFraction * Nclips
    Outputs:
        X_train: training set
        X_test: test set
        t_train: training labels
        t_test: test labels
    """


    ### WRITE YOUR CODE HERE - 5 MARKS

    
    test_samples = (int) (len(X)*testFraction )
    
    temp=np.arange(len(X))
    np.random.shuffle(temp)
    
    
    temp1 = temp[0: test_samples]
    temp2 = temp[test_samples: len(X)]
    
   
    
    X_test = np.reshape(X[temp1[0]],(-1,1))
    X_test = np.transpose(X_test)
    
    t_test = np.reshape(t[temp1[0]],(-1,1))
    t_test = np.transpose(t_test)
    for x in range(1,len(temp1)):
        i=temp1[x]
        a=np.reshape(X[i],(-1,1))
        a=np.transpose(a)
        b=np.reshape(t[i],(-1,1))
        b=np.transpose(b)
        X_test = np.concatenate((X_test,a))
        t_test = np.concatenate((t_test,b))
        
    X_train = np.reshape(X[temp2[0]],(-1,1))
    X_train = np.transpose(X_train)
    
    t_train = np.reshape(t[temp2[0]],(-1,1))
    t_train = np.transpose(t_train)
    for x in range(1,len(temp2)):
        i=temp2[x]
        a=np.reshape(X[i],(-1,1))
        a=np.transpose(a)
        b=np.reshape(t[i],(-1,1))
        b=np.transpose(b)
        X_train = np.concatenate((X_train,a))
        t_train = np.concatenate((t_train,b))
        
    return X_train, t_train, X_test, t_test

In [5]:
from glob import glob 
def readDir(dirname, Fs = 16000):
    
    '''
    This function reads all training samples, find thier power spectrogram, convert power spectrogram to mfcc features and 
    concatenate all frames of all samples along rows. Hence output matrix is frames X n_mfcc
    '''  


    no_samp_in_10_sec = 10* Fs
    files = glob(dirname + '/*.wav')
    x= np.ones((1,2))
    
    
    
    
    
    for f in files:
    
        
        samples = load_audio(f,Fs)
        n_fft = 1024
        hop_length = 512
        win_length = 1024
        X = np.abs(librosa.stft(samples, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann'))
        X = librosa.power_to_db(X**2,ref=np.max)
        
        temp =librosa.db_to_power(X)
        mel_spectrogram = librosa.feature.melspectrogram(y='none',S=temp, sr=16000)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
        
        
        mfccs = librosa.feature.mfcc(y='none',S=log_mel_spectrogram, n_mfcc=3, sr=16000,hop_length = hop_length, win_length = win_length ,n_fft=n_fft,window='hann')
        mfccs= mfccs[1:]
        #mfccs = librosa.feature.mfcc(y=samples, n_mfcc=2, sr=Fs,n_fft=1024,hop_length = 512, win_length = 1024)
        
        #delta_mfccs = librosa.feature.delta(mfccs)
        #delta2_mfccs = librosa.feature.delta(mfccs, order=2)
        #mfccs=np.concatenate((mfccs,delta_mfccs))
        #mfccs=np.concatenate((mfccs,delta2_mfccs))
        mfccs=np.array(mfccs)
        
        mfccs=np.transpose(mfccs)
        x=np.concatenate((x,mfccs))
    
    #print(k)
    
    return x[1:]


    return x 

In [6]:
def load_audio(filename, Fs = 16000):
    '''
    Inputs: 
        filename: (str) filename
        Fs: (int) sampling rate
    Output: 
        x: 1D np array 
    '''
    

    ### WRITE YOUR CODE HERE - 2 MARKS

    x,sr=librosa.load(filename,sr=Fs)
    x=np.array(x)
    
    return x

In [7]:
class GMM:
    def __init__(self,n_clusters ,n_iters = 1000):
        self.n_clusters = n_clusters
        self.n_iters = n_iters
        self.pi = [1/self.n_clusters for _ in range(self.n_clusters)]
    
    def gaussian(self, data ,mean, covariance):
        d = len(data)
        cov = np.linalg.det(covariance)
        temp = np.dot(np.dot((data-mean).T,np.linalg.inv(covariance)),(data-mean))
        return (2*np.pi)**(-d/2)*cov**(-1/2)*np.exp((-1/2)*temp)
    
    def fit(self,data):
        new_data = np.array_split(data,self.n_clusters)
        self.mean_arr = [np.mean(x,axis = 0) for x in new_data]
        self.covariance_mat = [np.cov(x.T) for x in new_data]
        
        
        for i in range(self.n_iters):
            self.r = np.zeros((len(data),self.n_clusters))
            for j in range(len(data)):
                for k in range(self.n_clusters):
                    self.r[j][k] = self.pi[k]*self.gaussian(data[j],self.mean_arr[k],self.covariance_mat[k])
                    
            for j in range(len(data)):
                for k in range(self.n_clusters):
                    self.r[j][k] /= np.sum(self.r[j])
            
            
            
            N = np.sum(self.r, axis = 0)
            self.mean_arr = np.zeros((self.n_clusters, len(data[0])))
            for k in range(self.n_clusters):
                for j in range(len(data)):
                    self.mean_arr[k] += self.r[j][k]*data[j]
            self.mean_arr = [(1/N[k])*self.mean_arr[k] for k in range(self.n_clusters)]
            
            self.covariance_mat = [np.zeros((len(data[0]), len(data[0]))) for k in range(self.n_clusters)]
            for j in range(self.n_clusters):
                self.covariance_mat[j] = np.cov(data.T, aweights = self.r[:,j], ddof = 0)
            self.covariance_mat = [1/N[k]*self.covariance_mat[k] for k in range(self.n_clusters)]
            
            self.pi = [N[k]/len(data) for k in range(self.n_clusters)]

    
    def predict(self,data):
        temp = []
        for i in range(len(data)):
            temp.append([self.gaussian(data[i], self.mean_arr[k],self.covariance_mat[k]) for k in range(self.n_clusters)])
        
        clusters = []
        for temp1 in temp:
            clusters.append(temp1.index(max(temp1)))
        return clusters

In [8]:
x_silence = readDir('C:/Users/HP/Documents/test/less_data_no-audio', 16000) 
x_speech =  readDir('C:/Users/HP/Documents/test/less_data_audio', 16000)
x_silence=x_silence[0:44]
x_speech=x_speech[0:44]

X = np.concatenate((x_silence, x_speech))
    
    # Create labels
y_silence = np.array([[1,0]]*len(x_silence))
y_speech = np.array([[0,1]]*len(x_speech))
Y = np.concatenate((y_silence, y_speech))

X_train,  y_train, X_test, y_test = splitData(X, Y, 0.2)


gmm1 = GMM(n_clusters = 2)
gmm1.fit(X_train)

In [13]:
x_music = readDir('C:/Users/HP/Documents/test/less_data_music', 16000) 
x_speech =  readDir('C:/Users/HP/Documents/test/less_data_speech', 16000)
x_music=x_music[57:101]
x_speech=x_speech[34:78]

X = np.concatenate((x_music, x_speech))
    
    # Create labels
y_music = np.array([[1,0]]*len(x_music))
y_speech = np.array([[0,1]]*len(x_speech))
Y = np.concatenate((y_music, y_speech))

X_train,  y_train, X_test, y_test = splitData(X, Y, 0.2)


gmm2 = GMM(n_clusters = 2)
gmm2.fit(X)


In [ ]:
#audio vs Silence

data = readDir('C:/Users/HP/Documents/test/unknown/', 16000) # This will work for single file in folder. For multiple files, WE can run this in a loop for every unknown audio file in folder
clusters=gmm1.predict(data)

temp=[]
for i in range(len(clusters)):
    if(clusters[i]==1):
        temp.append([0,1])
    else:
        temp.append([1,0])
pred=np.array(temp)

p=pred       

In [ ]:
#Time label prediction

k=0
j=0
sil=np.array([[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0]])
sp=np.array([[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0]])

def check(x):
    sum=0
    for i in range(0,len(x)):
        sum=sum+x[i][0]
    if(sum>=10):
        return 1
    else:
        return 0
    
    
    
prev=1

for i in range(0,len(p)-17):
    x=p[i:i+16]
    a=check(x)
    #print(a)
    if(a==1):
        if(prev==1):
            sil[j][1]=librosa.frames_to_time(i+22, sr=16000, hop_length=512, n_fft=1024)
        else:
            temp=i
            while(p[i][0]!=1):
                i+=1
            sp[k][1]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sil[j][0]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sil[j][1]=librosa.frames_to_time(temp+22, sr=16000, hop_length=512, n_fft=1024)
            k+=1
        
    
    else:
        if(prev==1):
            temp=i
            while(p[i][1]!=1):
                i+=1
            sil[j][1]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sp[k][0]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sp[k][1]=librosa.frames_to_time(temp+22, sr=16000, hop_length=512, n_fft=1024)
            j+=1
        
        else:
            sp[k][1]=librosa.frames_to_time(i+22, sr=16000, hop_length=512, n_fft=1024)
    prev=a      

    
sp  

In [ ]:
#Extracting audio portions

lis=[]
for i in range(len(sp)):
    if(sp[i][0]==0 and sp[i][1]==0):
        continue
    else:
        lis.append(np.array([sp[i][0],sp[i][1]]))
sp=np.array(lis)


lis=[]
for i in range(0,len(sp)):
    start= librosa.time_to_frames(sp[i][0], sr=16000, hop_length=512, n_fft=1024)
    end = librosa.time_to_frames(sp[i][1], sr=16000, hop_length=512, n_fft=1024)
    #print(start,end)
    lis.append(np.array(data[start:end]))

In [ ]:
#Frame labels

frame_pred= predict_framewise(gmm2,lis)
agg=predict_aggregate(frame_pred)  

for i in range(len(sp)):
    print(sp[i][0],sp[i][1], 'music' if (agg[i][0]==1) else  'speech')

In [17]:
np.save('C:/Users/HP/Documents/test/models/gmm_model1/mod1_mean.npy',gmm1.mean_arr)
np.save('C:/Users/HP/Documents/test/models/gmm_model1/mod1_cov.npy',gmm1.covariance_mat)

np.save('C:/Users/HP/Documents/test/models/gmm_model2/mod2_mean.npy',gmm2.mean_arr)
np.save('C:/Users/HP/Documents/test/models/gmm_model2/mod2_cov.npy',gmm2.covariance_mat)

In [14]:
gmm2.mean_arr

[array([113.76613714,  -3.06915856]), array([114.38953102,   6.48944227])]

In [15]:
gmm1.mean_arr

[array([123.07409293,  13.10343744]), array([116.99511814, -37.6705486 ])]